# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [145]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [829]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return (l_s - (np.sum(l_c * l_c / l_s, axis=1, keepdims=True))
                + r_s - (np.sum(r_c * r_c / r_s, axis=1, keepdims=True)))
    # Ваш код в 1 строчку

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -(np.sum((l_c * np.log2(((l_c / l_s))) / l_s)
                        + (r_c * np.log2(((r_c / r_s))) / r_s),
                        axis=1, keepdims=True))
    # Ваш код в 1 строчку

    def __misclass(self, l_c, l_s, r_c, r_s):
        return (l_s + r_s) - (np.max(l_c + r_c, axis=1, keepdims=True))
    # Ваш код в 1 строчку

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[0:int(np.sqrt(n_feature))])
    # Ваш код в 1 строчку

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[0:int(np.log2(n_feature))])
    # Ваш код в 1 строчку

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature, dtype=int)  # Ваш код

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        """
        Функция  __sort_samples(x, y) возвращает отсортированный массив х(в нем находятся значения n-ой фичи),
        каждому элементу которого соответствует определенный класс из массива y.
        Затем np.unique(y).shape[0] возвращает колличество разнообразных классов в массиве y
        """
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]

        # Что делает этот блок кода?
        """
        Выделяем область для анализа в массив splitted_sorted_y. Определяем индексы внутри области,
        на которых происходит смена класса. Эти значения содержаться в r_border_ids.
        Если все индексы в sorted_y одинаковы возвращаем бесконечную меру неопределенности( то есть работа завершена)
        """
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)

        if len(r_border_ids) == 0:
            return float('-Inf'), float('-Inf')

        # Что делает этот блок кода?
        """
        В eq_el_count записывается колличество одинаковых классов в каждом из кусочков
        splitted_sorted_y. В one_hot_code записывается матрица One Hot Encoding. Затем создаем матрицу
        class_increments, в которая содержит колличество одинаковых классов, которые идут друг за другом в массиве
        splitted_sorted_y.
        """
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split],
                                                                minlength=class_number)

        # Что делает этот блок кода?
        """
        l_class_count[i] и r_class_count[i] содержат колличество элементов принадлежащих классам слева и справ от
        i-ого элемента x соответственно. l_sizes[i] и r_sizes[i] содержат общее колличество элементов слева и справа
        от i-ого элемента x соответственно. Данные массивы используем для вычисления меры неопределенности.
        """
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        """
        Высчитывается мера неопределенности и берется минимальный индекс значения из массива для лучшего разбиения
        """
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        """
        Находим индекс элемента, при котором мера неопределенности наименьшая. Затем возвращаем ее и элемент,
        относительно которого будем разбивать корень дерева.
        """
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        (_, idx, counts) = np.unique(y, return_index=True, return_counts=True)
        clas_probs = counts / np.sum(counts)
        cl = y[idx[np.argmax(counts)]]

        if self.max_depth is not None and depth == self.max_depth:
            self.tree.update({node_id: np.array([self.LEAF_TYPE, cl, clas_probs[0]])})
            return

        if self.min_samples_split is not None and np.shape(y)[0] < self.min_samples_split:
            self.tree.update({node_id: np.array([self.LEAF_TYPE, cl, clas_probs[0]])})
            return

        features = np.shape(x)[1]
        n_feature = self.get_feature_ids(features)
        thresholds = np.zeros((features, 2), dtype=float)

        f_clas = np.min(y)
        y -= f_clas

        for i in n_feature:
            thresholds[i] = self.__find_threshold(x[:, i], y)

        y += f_clas
        if node_id != 0 and pred_f != -1:
            min_array = np.argsort(thresholds[:, 0])
            if n_feature[min_array[0]] == pred_f:
                min_arg = min_array[1]
            else:
                min_arg = min_array[0]
        else:
            min_arg = np.argmin(thresholds[:, 0])

        if np.isnan(thresholds[min_arg, 0]):
            self.tree.update({node_id: np.array([self.LEAF_TYPE, cl,
                                                 clas_probs[0]])})
            return

        a = np.array([self.NON_LEAF_TYPE, n_feature[min_arg], thresholds[min_arg, 1]])
        self.tree.update({node_id: a})

        x_l, x_r, y_l, y_r = self.__div_samples(x, y, n_feature[min_arg], thresholds[min_arg, 1])

        if np.shape(y_l)[0] == 0:
            self.tree.update({node_id: np.array([self.LEAF_TYPE, cl, clas_probs[0]])})
            return
        elif np.shape(y_r)[0] == 0:
            self.tree.update({node_id: np.array([self.LEAF_TYPE, cl, clas_probs[0]])})
            return

        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1, n_feature[min_arg])
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1, n_feature[min_arg])

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[int(feature_id)] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[int(feature_id)] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [698]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, criterion="entropy")
clf = DecisionTreeClassifier(min_samples_split=2)

In [699]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [700]:
%time clf.fit(X_train, y_train)

Wall time: 1.5 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [701]:
%time my_clf.fit(X_train, y_train)

Wall time: 48 ms


## Проверка качества работы на wine

In [702]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8917748917748917

In [703]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9474747474747475

## Подготовка данных Speed Dating Data 

In [763]:
# тут делаете то же самое, что и на семинаре https://github.com/stroykova/spheremailru/blob/master/2018-02/lecture_04_trees/pract-speed-dating-trees-proc.ipynb
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv', encoding='cp1251')

In [764]:
df = df.iloc[:, :97]
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)

df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att', 
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o','met_o'], 
             axis=1)

df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = df.drop(['field'], axis=1)

df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)

df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)

df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)

df = df.dropna(subset=['date'])

df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df = df.drop(['career'], axis=1)
df = df.drop(['sports','tvsports','exercise','dining','museums','art','hiking','gaming',
       'clubbing','reading','tv','theater','movies','concerts','music','shopping','yoga'], axis=1)
df = df.drop(['expnum'], axis=1)


df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)

df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
(df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100

df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
(df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100

df = df.drop(['temp_totalsum'], axis=1)

for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    
    df = df.drop(feat, axis=1)
df = df.drop(['wave'], axis=1)



In [765]:
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
        
df_female.columns = df_female.columns + '_f'

df_female = df_female.drop(['pid_f'], axis=1)

df_pair = df_male.join(df_female.set_index('iid_f'), 
                       on='pid', 
                       how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)

In [766]:
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

In [767]:
X_train.shape

(3599, 62)

In [768]:
X_test.shape

(400, 62)

## Проверка скорости работы на Speed Dating Data 

In [779]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, criterion="gini")
clf = DecisionTreeClassifier(min_samples_split=2)

In [780]:
# тут должен быть код типа %time clf.fit(X_train, y_train)
%time clf.fit(X_train, y_train)

Wall time: 108 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [781]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)
%time my_clf.fit(X_train, y_train)

Wall time: 865 ms


## Проверка качества работы на Speed Dating Data

In [782]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5497835497835498

In [783]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5638363438107429

## Задание 3

## Задание 4

In [785]:
model = DecisionTreeClassifier(max_depth=2)
model.fit(X_train, y_train)

features_names = df_pair.columns[1:]
df = pd.Series(index=features_names, data=model.feature_importances_)\
.sort_values(ascending=False)
df.head(10)

date        0.517202
fun1_1_f    0.343779
race        0.139019
amb3_1_f    0.000000
attr2_1     0.000000
intel1_1    0.000000
fun1_1      0.000000
amb1_1      0.000000
shar1_1     0.000000
sinc2_1     0.000000
dtype: float64

## Задание 5

In [788]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

In [795]:
def report(results, n_top=5):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

clf = RandomForestClassifier(n_estimators=20)
param_dist = {"max_depth": [8, None],
              "max_features": sp_randint(1, 11),
              "criterion": ["gini", "entropy"]}

n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)
random_search.fit(X_train, y_train)
report(random_search.cv_results_)

Model with rank: 1
Mean validation score: 0.843 (std: 0.001)
Parameters: {'criterion': 'entropy', 'max_depth': None, 'max_features': 7}

Model with rank: 2
Mean validation score: 0.841 (std: 0.002)
Parameters: {'criterion': 'entropy', 'max_depth': None, 'max_features': 7}

Model with rank: 2
Mean validation score: 0.841 (std: 0.002)
Parameters: {'criterion': 'gini', 'max_depth': None, 'max_features': 10}

Model with rank: 4
Mean validation score: 0.840 (std: 0.001)
Parameters: {'criterion': 'entropy', 'max_depth': 8, 'max_features': 9}

Model with rank: 5
Mean validation score: 0.840 (std: 0.002)
Parameters: {'criterion': 'entropy', 'max_depth': 8, 'max_features': 9}

Model with rank: 5
Mean validation score: 0.840 (std: 0.002)
Parameters: {'criterion': 'gini', 'max_depth': 8, 'max_features': 5}

